In [1]:
!pip install python-telegram-bot Application

  Using cached application-1.0.tar.gz (2.1 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached flask-3.1.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached flask_marshmallow-0.9.0-py2.py3-none-any.whl.metadata (6.4 kB)
  Using cached Flask_Migrate-2.3.0-py2.py3-none-any.whl.metadata (946 bytes)
  Using cached Flask-MySQLdb-0.2.0.tar.gz (2.1 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached Flask_REST

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      Traceback (most recent call last):
        File "C:\Python\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
        File "C:\Python\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Python\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\a.n.piskunov\AppData\Local\Temp\pip-build-env-wlfo0f__\overlay\Lib\site-packages\setuptools\build_meta.py", line 331, in get_requires_for_build_wheel
          return self._get_build_

In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import schedule
import time
import logging
from telegram import Bot, Update
from telegram.ext import Updater, CommandHandler, CallbackContext
import joblib  # Для загрузки ML-модели
from queue import Queue
from telegram.ext import Application, CommandHandler, ContextTypes
import pandas as pd
import sys
sys.path.append('c:\\Users\\a.n.piskunov\\Desktop\\gitlab\\input_hack_ekanam227')
from backend.tasks.date_parser import custom_date_parser
from backend.tasks.Lenta_parser import LentaNewsParser
from backend.tasks.Rbc_parser import rbc_parser
from backend.tasks.Agg_parser import NewsParser


MODELS = {
    'CHMF': joblib.load('c:\\Users\\a.n.piskunov\\Desktop\\gitlab\\input_hack_ekanam227\\backend\\tasks\\catboost_CHMF.pkl'),
    'GAZP': joblib.load('c:\\Users\\a.n.piskunov\\Desktop\\gitlab\\input_hack_ekanam227\\backend\\tasks\\catboost_GAZP.pkl'),
    'LKOH': joblib.load('c:\\Users\\a.n.piskunov\\Desktop\\gitlab\\input_hack_ekanam227\\backend\\tasks\\catboost_LKOH.pkl'),
    'T': joblib.load('c:\\Users\\a.n.piskunov\\Desktop\\gitlab\\input_hack_ekanam227\\backend\\tasks\\catboost_T.pkl')
}
model = SentenceTransformer('all-MiniLM-L6-v2')

# Конфигурация
TOKEN = "7855299161:AAE2CP_KMVw1YCfko6gwYsI07YQpzYZlekU"

DB_NAME = "../database.sqlite"
CHECK_INTERVAL = 1  # минут
name = 'No_time'

# Настройка логирования
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(name)

# Инициализация ML-модели (заглушка)
class NewsScoringModel:
    def __init__(self):
        self.tickers = ['CHMF', 'GAZP', 'LKOH', 'T']


    def predict(self, df) -> dict:
        """Предсказание для всех тикеров"""
        df['emb'] = df['text'].apply(lambda x: model.encode(x))
        X = np.vstack(df['emb'].values)
        return {
            ticker: MODELS[ticker].predict_proba(X)[0][1]
            for ticker in self.tickers
        }


def get_all_users():
    conn = sqlite3.connect(DB_NAME)
    c = conn.cursor()
    c.execute("SELECT chat_id FROM users")
    users = [row[0] for row in c.fetchall()]
    conn.close()
    return users


# Парсер новостей
def parse_news():
    try:
        parser = NewsParser(csv_file='news_data.csv')
        new_data = parser.parse_new()
        
        news_items = []
        for _, row in new_data.iterrows():
            news_id = f"{row['source']}-{hash(row['url'])}"
            news_items.append({
                'id': news_id,
                'title': row['title'],
                'text': row['text'],
                'link': row['url'],
                'date': row['date']
            })
        return news_items
    except Exception as e:
        logger.error(f"Ошибка парсинга: {str(e)}")
        return []
    
# Обработчик команд бота
def start(update: Update, context: CallbackContext):
    user = update.effective_user
    update.message.reply_text(f"Привет {user.first_name}! Теперь ты будешь получать важные новости.")

# Рассылка новостей
def send_news_to_users(news_item):
    bot = Bot(token=TOKEN)
    users = get_all_users()
    message = f"🚨 Важная новость! 🚨\n\n{news_item['title']}\n\n{news_item['link']}"
    
    for chat_id in users:
        try:
            bot.send_message(chat_id=chat_id, text=message)
            logger.info(f"Отправлено пользователю {chat_id}")
        except Exception as e:
            logger.error(f"Ошибка отправки для {chat_id}: {str(e)}")


# Основная задача для планировщика
def monitoring_task():
    logger.info("Запуск проверки новостей...")
    model = NewsScoringModel()
    
    news_items = parse_news()  # Теперь парсим все источники сразу
    logger.info(f"Всего найдено {len(news_items)} новых новостей")
        
    for item in news_items:
            
        score = model.predict(item['text'])
        
        if score > 0.6:
            logger.info(f"Важная новость! Скор: {score:.4f} - {item['title']}")
            send_news_to_users(item)

# Инициализация
async def main():
    # Инициализация приложения
    application = Application.builder().token(TOKEN).build()
    
    # Регистрация обработчиков
    application.add_handler(CommandHandler("start", start))
    
    # Запуск бота
    await application.initialize()
    await application.start()
    
    # Запуск планировщика в отдельном потоке
    from threading import Thread
    Thread(target=run_scheduler).start()
    
    # Ожидание остановки
    await application.stop()
    await application.shutdown()

def run_scheduler():
    schedule.every(CHECK_INTERVAL).minutes.do(monitoring_task)
    logger.info(f"Сервис запущен. Проверка каждые {CHECK_INTERVAL} минут.")
    while True:
        schedule.run_pending()
        time.sleep(1)

if __name__ == "__main__":
    import asyncio
    await main()


2025-06-08 12:48:04,627 - httpx - INFO - HTTP Request: POST https://api.telegram.org/bot7855299161:AAE2CP_KMVw1YCfko6gwYsI07YQpzYZlekU/getMe "HTTP/1.1 200 OK"
2025-06-08 12:48:04,627 - telegram.ext.Application - INFO - Application started
2025-06-08 12:48:04,642 - No_time - INFO - Сервис запущен. Проверка каждые 1 минут.
2025-06-08 12:48:04,644 - telegram.ext.Application - INFO - Application is stopping. This might take a moment.
2025-06-08 12:48:04,644 - No_time - INFO - Запуск проверки новостей...
2025-06-08 12:48:04,644 - telegram.ext.Application - INFO - Application.stop() complete


Checking: https://lenta.ru/news/2025/06/07/page/1/


Error processing https://lenta.ru/news/2025/06/07/page/1/: HTTPSConnectionPool(host='lenta.ru', port=443): Max retries exceeded with url: /news/2025/06/07/page/1/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002341871B5D0>, 'Connection to lenta.ru timed out. (connect timeout=None)'))
Checking: https://lenta.ru/news/2025/06/08/page/1/
Checking: https://lenta.ru/news/2025/06/08/page/2/
Checking: https://lenta.ru/news/2025/06/08/page/3/
Checking: https://lenta.ru/news/2025/06/08/page/4/
Checking: https://lenta.ru/news/2025/06/08/page/5/


2025-06-08 12:51:49,867 - No_time - INFO - Всего найдено 279 новых новостей
Exception in thread Thread-56 (run_scheduler):
Traceback (most recent call last):
  File "c:\Python\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Python\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Python\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\a.n.piskunov\AppData\Local\Temp\ipykernel_20524\780865446.py", line 152, in run_scheduler
  File "c:\Python\Lib\site-packages\schedule\__init__.py", line 854, in run_pending
    default_scheduler.run_pending()
  File "c:\Python\Lib\site-packages\schedule\__init__.py", line 101, in run_pending
    self._run_job(job)
  File "c:\Python\Lib\site-packages\schedule\__init__.py", line 173, in _run_job
    ret = job.run()
          ^^^^^^^^^
  File "c:\Python\Lib\site-packages\schedule\__init__.py", line 691, in run
    ret = s

Finish
Добавлено 279 новых записей
